In [18]:
import numpy as np
from PGM_PyLib.MRF import RMRFwO as mrf

In [19]:
# Define the states (representing "tree", "glass", "car", and "cup")
s = [0, 1, 2, 3]  # States corresponding to ["tree", "glass", "car", "cup"]

# Initialize the RMRF as a vector of length 3 (one component for each bounding box)
r = np.zeros(3, dtype=int)  # Initial RMRF values (neutral state, "tree")
print("Initial RMRF\n", r)

# Observed bounding box probability distributions
# 3 bounding boxes with probability distributions over the 4 states
bounding_box_probabilities = np.array([
    [0.2, 0.5, 0.2, 0.1],  # Probability distribution for bounding box 1
    [0.4, 0.1, 0.3, 0.2],  # Probability distribution for bounding box 2
    [0.25, 0.25, 0.25, 0.25]  # Probability distribution for bounding box 3 (uniform)
])

print("\nObservation\n", bounding_box_probabilities)

Initial RMRF
 [0 0 0]

Observation
 [[0.2  0.5  0.2  0.1 ]
 [0.4  0.1  0.3  0.2 ]
 [0.25 0.25 0.25 0.25]]


In [20]:
# Generate the 4x4 relational probability matrix (for the 4 states)
def generate_symmetric_matrix(n):
    """Generate an n x n symmetric matrix for 0-based indexing."""
    mat = np.random.rand(n, n)  # Generate random values
    mat = (mat + mat.T) / 2  # Make the matrix symmetric
    np.fill_diagonal(mat, 1)  # Fill the diagonal with 1s for self-relations
    return mat

# Relational probability matrix for the four states
relational_probabilities = generate_symmetric_matrix(4)
print("\nRelational Probability Matrix (0-based indexing):")
print(relational_probabilities)


Relational Probability Matrix (0-based indexing):
[[1.         0.59983551 0.8504227  0.49961806]
 [0.59983551 1.         0.42290321 0.7223121 ]
 [0.8504227  0.42290321 1.         0.81972281]
 [0.49961806 0.7223121  0.81972281 1.        ]]


In [21]:
# Define psi and Uf functions
def psi(state_i, state_j):
    """
    Return the probability of transitioning from state_i to state_j
    based on the relational probability matrix (0-based indexing).
    """
    return relational_probabilities[state_i, state_j]

def Uf(rmrf, observation, row, col):
    """
    Local energy function for a cell at position (row, col).
    
    It calculates the energy based on the current state of the cell
    and its neighbors, using the psi function and observation data.
    
    Parameters:
    - rmrf: 2D numpy array, current RMRF matrix (states should be 0-based)
    - observation: bounding box probability distributions
    - row: int, row index of the cell
    - col: int, column index of the cell

    Returns:
    - float, the calculated energy for the cell at (row, col)
    """
    current_state = rmrf[row, col]  # The state is 0-based
    energy = 0.0

    # Use observation data (bounding box probabilities)
    # Assuming bounding boxes are mapped to row/col based on some external logic
    # For simplicity, we'll assume the observation influences the cell's state
    if row < len(observation):
        obs_prob = observation[row][current_state]  # Probability for the current state
        energy -= np.log(obs_prob)  # Factor in observation likelihood
    
    # Define the neighbors (assuming a 4-neighbor grid)
    neighbors = [(row-1, col), (row+1, col), (row, col-1), (row, col+1)]
    
    for n_row, n_col in neighbors:
        if 0 <= n_row < rmrf.shape[0] and 0 <= n_col < rmrf.shape[1]:
            neighbor_state = rmrf[n_row, n_col]
            prob = psi(current_state, neighbor_state)  # Access relational probabilities
            energy -= np.log(prob)  # Energy is -log(psi)
    
    return energy

In [22]:
# Potential function (pairwise potentials)
def psi(state_i, state_j):
    """
    Return the probability of transitioning from state_i to state_j
    based on the relational probability matrix.
    """
    return relational_probabilities[state_i, state_j]  # Using global matrix

# Local energy function (Uf) to be passed to the inference method
def Uf(rmrf, observation, row, col):
    """
    Local energy function for a cell at position (row, col).
    
    It calculates the energy based on the current state of the cell
    and its neighbors, using the psi function.
    
    Parameters:
    - rmrf: 2D numpy array, current RMRF matrix (states should be 0-based)
    - observation: observation object (optional, not used here)
    - row: int, row index of the cell
    - col: int, column index of the cell

    Returns:
    - float, the calculated energy for the cell at (row, col)
    """
    current_state = rmrf[row, col]  # The state is 0-based
    energy = 0.0
    
    # Define the neighbors (assuming a 4-neighbor grid)
    neighbors = [(row-1, col), (row+1, col), (row, col-1), (row, col+1)]
    
    for n_row, n_col in neighbors:
        if 0 <= n_row < rmrf.shape[0] and 0 <= n_col < rmrf.shape[1]:
            neighbor_state = rmrf[n_row, n_col]
            prob = psi(current_state, neighbor_state)  # Accessing psi using global matrix
            energy -= np.log(prob)  # Energy is -log(psi)
    
    return energy

In [23]:
# Create an instance of RMRFwO and run inference
# The states are [0, 1, 2, 3] corresponding to the object classification classes
mr = mrf(s, r, bounding_box_probabilities)  # Use the bounding box probabilities as the observation

# Run inference using the updated Uf function
result = mr.inference(Uf=Uf, maxIterations=100, Temp=1.0, tempReduction=1.0, optimal="MAP")

print("\nFinal RMRF after inference:")
print(result)

NameError: Error: rmrf has to be a numpy matrix of shape (x,y)